Шифр сдвига (Цезаря)

In [1]:
def load_file(file_name):
    """Загрузка данных из файла."""
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            return file.read().strip()
    except FileNotFoundError:
        raise Exception(f"Файл {file_name} не найден.")

def validate_alphabet(alphabet): # АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ .,!?
    """Проверка корректности алфавита."""
    if len(alphabet) != len(set(alphabet)):
        raise Exception("Алфавит содержит дублирующиеся символы.")
    
def read_message(file_path):
    """Чтение и вывод содержимого файла."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    print(content)

def validate_caesar_key(key):
    """Проверка корректности ключа."""
    if not key.isdigit():
        raise Exception("Ключ должен быть числом.")
    return int(key)

def shift_character(char, alphabet, shift_amount):
    """Сдвиг символа на указанное количество позиций."""
    if char in alphabet:
        index = (alphabet.index(char) + shift_amount) % len(alphabet)
        return alphabet[index]
    return char  # Возвращаем символ без изменений, если он не в алфавите

def caesar_cipher(text, alphabet, key, operation):
    """Шифрование или расшифрование текста с использованием шифра Цезаря."""
    if operation not in ["encrypt", "decrypt"]:
        raise Exception("Некорректная операция. Выберите 'encrypt' или 'decrypt'.")
    
    # Устанавливаем направление сдвига
    shift = key if operation == "encrypt" else -key
    
    # Применяем сдвиг к каждому символу
    return ''.join(shift_character(char, alphabet, shift) for char in text)


In [2]:
# Загрузка данных из файлов
alphabet = load_file('alphabet.txt')
validate_alphabet(alphabet)

In [148]:
try:
    message = load_file('in.txt').upper()
    #message = load_file('crypt.txt')
    key = load_file('caesar_key.txt')
    key = validate_caesar_key(key)
    
    # Указание операции и криптосистемы
    operation = input("Введите операцию (encrypt/decrypt): ").strip().lower()
    
    # Выполнение операции
    result = caesar_cipher(message, alphabet, key, operation)
    
    # Запись результата в файл
    output_file = 'crypt.txt' if operation == 'encrypt' else 'decrypt.txt'
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(result)
    
    print(f"Операция {operation} прошла успешно. Результат записан в {output_file}.")
    print("\n" + message + "\n =======>")
    read_message(output_file)

except Exception as e:
    print(f"Ошибка: {e}")

Операция encrypt прошла успешно. Результат записан в crypt.txt.

ШТИРЛИЦ ВСЮ НОЧЬ ТОПИЛ КАМИН. НА УТРО КАМИН УТОНУЛ.
 =======>
ЫХЛУОЛЩ!ЕФ.!РСЪЯ!ХСТЛО!НГПЛР?!РГ!ЦХУС!НГПЛР!ЦХСРЦО?


Афинный шифр

In [20]:
import random

def gcd(a, b):
    """Вычисление наибольшего общего делителя."""
    while b:
        a, b = b, a % b
    return a

def mod_inverse(a, m):
    """Вычисление мультипликативной обратной величины."""
    a = a % m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    raise Exception("Обратного элемента не существует.")

def generate_affine_key(alphabet):
    """Генерация ключа для аффинного шифра."""
    M = len(alphabet)
    while True:
        k1 = random.choice(alphabet)
        k2 = random.choice(alphabet)
        k1_index = alphabet.index(k1)
        
        if gcd(k1_index, M) == 1:
            return k1 + k2

def validate_affine_key(key, alphabet_length):
    """Проверка корректности ключа."""
    if len(key) != 2:
        raise Exception("Ключ должен состоять из двух символов.")
    k1, k2 = alphabet.index(key[0]), alphabet.index(key[1])
    if gcd(k1, alphabet_length) != 1:
        raise Exception("Числовое представление k1 и M не взаимно просты.")
    return k1, k2

def affine_encrypt(char, alphabet, k1, k2):
    """Шифрование одного символа аффинным шифром."""
    if char in alphabet:
        index = alphabet.index(char)
        new_index = (k1 * index + k2) % len(alphabet)
        return alphabet[new_index]
    return char

def affine_decrypt(char, alphabet, k1, k2):
    """Расшифрование одного символа аффинным шифром."""
    if char in alphabet:
        index = alphabet.index(char)
        k1_inv = mod_inverse(k1, len(alphabet))
        new_index = (k1_inv * (index - k2)) % len(alphabet)
        return alphabet[new_index]
    return char

def affine_cipher(text, alphabet, k1, k2, operation):
    """Шифрование или расшифрование текста с использованием аффинного шифра."""
    if operation not in ["encrypt", "decrypt"]:
        raise Exception("Некорректная операция. Выберите 'encrypt' или 'decrypt'.")
    
    if operation == "encrypt":
        return ''.join(affine_encrypt(char, alphabet, k1, k2) for char in text)
    else:
        return ''.join(affine_decrypt(char, alphabet, k1, k2) for char in text)


In [153]:
key = generate_affine_key(alphabet)
print(f"Сгенерированный ключ: {key}")
with open("affine_key.txt", 'w') as f:
        f.write(key)

Сгенерированный ключ: ФЦ


In [21]:
try:
    
    message = load_file('in.txt').upper()
    #message = load_file('crypt.txt')
    key = load_file('affine_key.txt')
    k1, k2 = validate_affine_key(key, len(alphabet))
    
    # Указание операции
    operation = input("Введите операцию (encrypt/decrypt): ").strip().lower()
    
    # Выполнение операции
    result = affine_cipher(message, alphabet, k1, k2, operation)
    
    # Запись результата в файл
    output_file = 'crypt.txt' if operation == 'encrypt' else 'decrypt.txt'
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(result)
    
    print(f"Операция {operation} прошла успешно. Результат записан в {output_file}.")
    print("\n" + message + "\n =======>")
    read_message(output_file)

except Exception as e:
    print(f"Ошибка: {e}")   

Ошибка: Числовое представление k1 и M не взаимно просты.


Шифр простой замены

In [17]:
import random

def generate_substitution_key(alphabet):
    """Генерация случайного ключа для шифра простой замены."""
    shuffled_alphabet = list(alphabet)
    random.shuffle(shuffled_alphabet)
    return ''.join(shuffled_alphabet)

def validate_substitution_key(key, alphabet):
    """Проверка корректности ключа."""
    if len(key) != len(alphabet) or set(key) != set(alphabet):
        raise Exception("Ключ должен содержать все символы алфавита один раз в случайном порядке.")

def substitution_encrypt_decrypt(text, alphabet, key, operation):
    """Шифрование или расшифрование текста с использованием шифра простой замены."""
    if operation not in ["encrypt", "decrypt"]:
        raise Exception("Некорректная операция. Выберите 'encrypt' или 'decrypt'.")
    
    # Создание словарей для замены
    if operation == "encrypt":
        substitution_dict = dict(zip(alphabet, key))
    else:
        substitution_dict = dict(zip(key, alphabet))
    
    # Замена символов по словарю
    return ''.join(substitution_dict.get(char, char) for char in text)


In [19]:
try:
    #message = load_file('in.txt').upper()
    message = load_file('crypt.txt')
    
    # Генерация или загрузка ключа
    use_generated_key = input("Сгенерировать новый ключ? (yes/no): ").strip().lower()
    if use_generated_key == "yes":
        key = generate_substitution_key(alphabet)
        with open('substitution_key.txt', 'w', encoding='utf-8') as file:
            file.write(key)
        print(f"Сгенерированный ключ: {key}\n")
    else:
        key = load_file('substitution_key.txt')
        validate_substitution_key(key, alphabet)
    
    # Указание операции
    operation = input("Введите операцию (encrypt/decrypt): ").strip().lower()
    
    # Выполнение операции
    result = substitution_encrypt_decrypt(message, alphabet, key, operation)
    
    # Запись результата в файл
    output_file = 'crypt.txt' if operation == 'encrypt' else 'decrypt.txt'
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(result)
    
    print(f"Операция {operation} прошла успешно. Результат записан в {output_file}.")
    print("\n" + message + "\n =======>")
    read_message(output_file)

except Exception as e:
    print(f"Ошибка: {e}")

Операция decrypt прошла успешно. Результат записан в decrypt.txt.

ЫХПЁНХОШЖНЧГЮЁОК
 =======>
НЕДОВЕРЯЙВИКТОРУ


Шифр Хилла

In [3]:
import numpy as np

# Чтение ключа из файла
def read_hill_key(file_path, alphabet):
    """Чтение ключа из файла и формирование ключевой матрицы."""
    with open(file_path, 'r', encoding='utf-8') as file:
        key = file.read().strip()
    
    # Проверка корректности ключа
    if len(key) != 4 or any(char not in alphabet for char in key):
        raise ValueError("Некорректный ключ. Должен содержать 4 символа из алфавита.")
    
    # Формирование матрицы 2x2 из индексов символов ключа
    key_indices = [alphabet.index(char) for char in key]
    key_matrix = np.array([[key_indices[0], key_indices[1]], [key_indices[2], key_indices[3]]])
    
    # Проверка обратимости матрицы
    if np.linalg.det(key_matrix) == 0:
        raise ValueError("Матрица ключа не обратима.")
    
    return key_matrix

def mod_inv(a, m):
    """Нахождение модульной инверсии числа a по модулю m."""
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def inverse_matrix(matrix, modulus):
    """Получение обратной матрицы по модулю."""
    det = int(np.round(np.linalg.det(matrix)))
    det_inv = mod_inv(det, modulus)
    if det_inv is None:
        raise ValueError("Обратная матрица не существует.")
    matrix_mod_inv = (det_inv * np.round(det * np.linalg.inv(matrix)).astype(int)) % modulus
    return matrix_mod_inv

def prepare_text(text, alphabet, block_size):
    """Подготовка текста: преобразование в индексы и дополнение до кратности."""
    digits = [alphabet.index(char) for char in text if char in alphabet]
    if len(digits) % block_size != 0:
        digits += [0] * (block_size - len(digits) % block_size)
    return digits

def hill_encrypt(plaintext, key_matrix, alphabet):
    """Шифрование текста с использованием ключевой матрицы."""
    modulus = len(alphabet)
    block_size = key_matrix.shape[0]
    digits = prepare_text(plaintext, alphabet, block_size)
    
    blocks = [digits[i:i+block_size] for i in range(0, len(digits), block_size)]
    ciphertext = []
    
    for block in blocks:
        vector = np.array(block)
        encrypted_vector = np.dot(key_matrix, vector) % modulus
        ciphertext.extend(encrypted_vector)
    
    return ''.join(alphabet[int(idx)] for idx in ciphertext)

def hill_decrypt(ciphertext, key_matrix, alphabet):
    """Расшифрование текста с использованием обратной ключевой матрицы."""
    modulus = len(alphabet)
    block_size = key_matrix.shape[0]
    digits = [alphabet.index(char) for char in ciphertext]
    
    inverse_key_matrix = inverse_matrix(key_matrix, modulus)
    blocks = [digits[i:i+block_size] for i in range(0, len(digits), block_size)]
    plaintext = []
    
    for block in blocks:
        vector = np.array(block)
        decrypted_vector = np.dot(inverse_key_matrix, vector) % modulus
        plaintext.extend(decrypted_vector)
    
    return ''.join(alphabet[int(idx)] for idx in plaintext)

In [5]:
# Загрузка данных из файлов
alphabet = load_file('alphabet.txt')
validate_alphabet(alphabet)

# Обработка операций
operation = input("Введите операцию (encrypt/decrypt): ").strip().lower()

try:
    key_matrix = read_hill_key('hill_key.txt', alphabet)
    #message = load_file('in.txt').upper()
    message = load_file('crypt.txt')
    
    if operation == 'encrypt':
        result = hill_encrypt(message, key_matrix, alphabet)
        output_file = 'crypt.txt'
    elif operation == 'decrypt':
        result = hill_decrypt(message, key_matrix, alphabet)
        output_file = 'decrypt.txt'
    else:
        print("Некорректная операция.")
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(result)
    
    print(f"Операция {operation} прошла успешно. Результат записан в {output_file}.")
    print("\n" + message + "\n =======>")
    read_message(output_file)

except Exception as e:
    print(f"Ошибка: {e}")

Операция decrypt прошла успешно. Результат записан в decrypt.txt.

КЁИЩМФШСЭУЪЩРЬООМНВЮНФФЁШМЛЦЦЬЯНЛОФ!ЦЙ!К?Х?ИЕБДЬЛЯС.
 =======>
ШТИРЛИЦ ВСЮ НОЧЬ ТОПИЛ КАМИН. НА УТРО КАМИН УТОНУЛ.А


Шифр перестановки

In [13]:
def read_permutation_key(file_path, alphabet):
    """Чтение ключа из файла и проверка его корректности."""
    with open(file_path, 'r', encoding='utf-8') as file:
        key = file.read().strip()
    
    # Проверка корректности ключа
    if len(set(key)) != len(key) or any(char not in alphabet for char in key):
        raise ValueError("Некорректный ключ. Должен содержать уникальные символы из алфавита.")
    
    return key

def generate_permutation_key(alphabet, max_length = len(alphabet)):
    """Генерирует ключ для шифра перестановки."""
    key_length = random.randint(2, max_length)
    key = ''.join(random.sample(alphabet, key_length))
    return key

def prepare_text(text, key_length, padding_char):
    """Дополнение текста до кратности длине ключа."""
    padding_size = (key_length - len(text) % key_length) % key_length
    return text + padding_char * padding_size

def permutation_encrypt(plaintext, key, alphabet):
    """Шифрование текста с использованием перестановочного шифра."""
    key_length = len(key)
    padded_text = prepare_text(plaintext, key_length, alphabet[-1])
    ciphertext = ''

    # Определение порядка столбцов для перестановки
    indices = sorted(range(len(key)), key=lambda i: alphabet.index(key[i]))
    
    # Перестановка по блокам
    for i in range(0, len(padded_text), key_length):
        block = padded_text[i:i+key_length]
        ciphertext += ''.join(block[j] for j in indices)
    
    return ciphertext

def permutation_decrypt(ciphertext, key, alphabet):
    """Расшифрование текста с использованием перестановочного шифра."""
    key_length = len(key)
    plaintext = ''
    
    # Определение порядка столбцов для обратной перестановки
    indices = sorted(range(len(key)), key=lambda i: alphabet.index(key[i]))
    reverse_indices = [0] * key_length
    for i, idx in enumerate(indices):
        reverse_indices[idx] = i

    # Обратная перестановка по блокам
    for i in range(0, len(ciphertext), key_length):
        block = ciphertext[i:i+key_length]
        plaintext += ''.join(block[j] for j in reverse_indices)
    
    return plaintext


In [187]:
alphabet = load_file('alphabet.txt')
key = generate_permutation_key(alphabet)
with open("permutation_key.txt", 'w') as f:
        f.write(key)
print(key)

ФЗЛСЩВ?УАЧНЫИ


In [15]:
# Загрузка данных из файлов
alphabet = load_file('alphabet.txt')
validate_alphabet(alphabet)

# Обработка операций
operation = input("Введите операцию (encrypt/decrypt): ").strip().lower()

try:
    key = read_permutation_key('permutation_key.txt', alphabet)
    message = load_file('in.txt').upper()
    #message = load_file('crypt.txt')
    
    if operation == 'encrypt':
        result = permutation_encrypt(message, key, alphabet)
        output_file = 'crypt.txt'
    elif operation == 'decrypt':
        result = permutation_decrypt(message, key, alphabet)
        output_file = 'decrypt.txt'
    else:
        print("Некорректная операция.")
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(result)
    
    print(f"Операция {operation} прошла успешно. Результат записан в {output_file}.")
    print("\n" + message + "\n =======>")
    read_message(output_file)

except Exception as e:
    print(f"Ошибка: {str(e)}")

Ошибка: Некорректный ключ. Должен содержать уникальные символы из алфавита.


Шифр Виженера

In [6]:
def read_vigenere_key(file_path, alphabet):
    """Чтение ключа из файла и проверка его корректности."""
    with open(file_path, 'r', encoding='utf-8') as file:
        key = file.read().strip()
    
    # Проверка корректности ключа
    if any(char not in alphabet for char in key):
        raise ValueError("Некорректный ключ. Все символы должны быть из алфавита.")
    
    return key

def vigenere_encrypt(plaintext, key, alphabet):
    """Шифрование текста с использованием шифра Виженера."""
    key_length = len(key)
    alphabet_length = len(alphabet)
    ciphertext = []

    for i, char in enumerate(plaintext):
        if char in alphabet:
            # Получаем индексы символа текста и соответствующего символа ключа
            text_index = alphabet.index(char)
            key_index = alphabet.index(key[i % key_length])
            # Вычисляем индекс зашифрованного символа
            cipher_index = (text_index + key_index) % alphabet_length
            ciphertext.append(alphabet[cipher_index])
        else:
            ciphertext.append(char)  # Игнорируем символы, не входящие в алфавит

    return ''.join(ciphertext)

def vigenere_decrypt(ciphertext, key, alphabet):
    """Расшифрование текста с использованием шифра Виженера."""
    key_length = len(key)
    alphabet_length = len(alphabet)
    plaintext = []

    for i, char in enumerate(ciphertext):
        if char in alphabet:
            # Получаем индексы символа текста и соответствующего символа ключа
            cipher_index = alphabet.index(char)
            key_index = alphabet.index(key[i % key_length])
            # Вычисляем индекс расшифрованного символа
            text_index = (cipher_index - key_index) % alphabet_length
            plaintext.append(alphabet[text_index])
        else:
            plaintext.append(char)  # Игнорируем символы, не входящие в алфавит

    return ''.join(plaintext)

In [8]:
# Обработка операций
operation = input("Введите операцию (encrypt/decrypt): ").strip().lower()

try:
    key = read_vigenere_key('vigenere_key.txt', alphabet).upper()
    message = load_file('in.txt').upper()
    #message = load_file('crypt.txt')
    
    if operation == 'encrypt':
        result = vigenere_encrypt(message, key, alphabet)
        output_file = 'crypt.txt'
    elif operation == 'decrypt':
        result = vigenere_decrypt(message, key, alphabet)
        output_file = 'decrypt.txt'
    else:
        print("Некорректная операция.")
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(result)
    
    print(f"Операция {operation} прошла успешно. Результат записан в {output_file}.")
    print("\n" + message + "\n =======>")
    read_message(output_file)

except Exception as e:
    print(f"Ошибка: {str(e)}")

Операция encrypt прошла успешно. Результат записан в crypt.txt.

ЧЧЧЧЧЧЧЧЧЧЧЧЧЧЧЧЧЧЧЧЧЧ
 =======>
КЕ Г! ИКЕ Г! ИКЕ Г! ИК
